In [7]:
!pip install pyspark==3.3.1

You should consider upgrading via the '/home/aluno/Documents/curso2/venv/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


23/12/15 01:33:20 WARN Utils: Your hostname, JulioPC resolves to a loopback address: 127.0.1.1; using 172.27.58.37 instead (on interface eth0)
23/12/15 01:33:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/15 01:33:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/15 01:33:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.json("../../datalake/Bronze/twitter_datascience")

In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{47, 8, 2023-12-...|{[{2023-12-11T14:...|{1234567890abcdef}|  2023-12-11|
|[{19, 95, 2023-12...|{[{2023-12-11T21:...|{1234567890abcdef}|  2023-12-11|
|[{73, 29, 2023-12...|{[{2023-12-11T07:...|{1234567890abcdef}|  2023-12-11|
|[{11, 13, 2023-12...|{[{2023-12-11T21:...|{1234567890abcdef}|  2023-12-11|
|[{66, 26, 2023-12...|{[{2023-12-11T15:...|              null|  2023-12-11|
|[{99, 40, 2023-12...|{[{2023-12-09T17:...|{1234567890abcdef}|  2023-12-09|
|[{81, 20, 2023-12...|{[{2023-12-09T08:...|              null|  2023-12-09|
|[{9, 45, 2023-12-...|{[{2023-12-12T10:...|              null|  2023-12-12|
|[{75, 64, 2023-12...|{[{2023-12-13T17:...|              null|  2023-12-13|
|[{97, 77, 2023-12...|{[{2023-12-14T05:...|              null|  2023-12-14|
|[{97, 27, 2

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [7]:
from pyspark.sql import functions as f

In [8]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)


In [9]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{47, 8, 2023-12-1...|
|{5, 90, 2023-12-1...|
|{28, 18, 2023-12-...|
|{69, 99, 2023-12-...|
|{39, 45, 2023-12-...|
|{91, 75, 2023-12-...|
|{3, 69, 2023-12-1...|
|{52, 87, 2023-12-...|
|{17, 32, 2023-12-...|
|{33, 70, 2023-12-...|
|{19, 95, 2023-12-...|
|{81, 99, 2023-12-...|
|{85, 20, 2023-12-...|
|{24, 30, 2023-12-...|
|{93, 83, 2023-12-...|
|{80, 50, 2023-12-...|
|{48, 23, 2023-12-...|
|{61, 81, 2023-12-...|
|{69, 16, 2023-12-...|
|{27, 37, 2023-12-...|
+--------------------+


In [10]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)


In [11]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [12]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       47|              8|2023-12-11T11:50:...| 51|        92|         45|         38|           39|Tweet fictício cr...|
|        5|             90|2023-12-11T18:09:...| 65|        38|         53|         44|          100|Outro tweet fictí...|
|       28|             18|2023-12-11T05:19:...| 72|        80|         82|         67|           87|Este é um tweet f...|
|       69|             99|2023-12-11T19:30:...| 63|        76|          7|         76|           16|Um terceiro tweet...|
|       39|             45|2023-12-11T18:20:...| 72|        41|         30|         28|           80|Tweet fictício ge...|
+---------+-----

In [13]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [14]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)


In [15]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [16]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-12-11T14:20:...| 40|User 1|   user1|
|2023-12-11T12:27:...| 43|User 2|   user2|
|2023-12-11T00:48:...| 95|User 3|   user3|
|2023-12-11T07:41:...| 74|User 4|   user4|
|2023-12-11T20:33:...|  8|User 5|   user5|
+--------------------+---+------+--------+


In [18]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')